# Sieci neuronowe i Deep Learning
# Temat 7: Mini projekt

## Zadanie 7.1

Przeanalizować poniższy projekt analizy danych (budowa sieci do predykcji zużycia paliwa) i uzupełnić brakujące fragmenty (zaznaczone w kodzie przez ```#####```).

## Zadanie 7.2*

Potraktować zbiór testowy z powyższej analizy jako zbiór walidacyjny i na jakiego podstawie dobrać optymalne hiperparametry rozważanego modelu.

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from IPython.display import Image

/Users/jedrzejsarna/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Predykcja zużycia paliwa przez samochód (w milach na galon: MPG)

Analiza przeprowadzona zostanie w oparciu o zbiór danych
*Auto MPG* (dane i opis: https://archive.ics.uci.edu/ml/datasets/auto+mpg).

In [ ]:
Image(filename='autoMPGdataset.png', width=700)

Wczytanie danych:

In [3]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

df = pd.read_csv(url, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)

df.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


Usuwamy wiersze z brakami:

In [4]:
print(df.isna().sum())

df = df.dropna()    ###########
df = df.reset_index(drop=True)
df.tail()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1
391,31.0,4,119.0,82.0,2720.0,19.4,82,1


Tworzymy zbiór treningowy (80% całego zbioru danych) i zbiór testowy:

In [5]:
import sklearn
import sklearn.model_selection


df_train, df_test = sklearn.model_selection.train_test_split(df, train_size=0.8)
train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,313.0,23.094569,7.492415,9.0,16.9,22.3,28.1,46.6
Cylinders,313.0,5.507987,1.741490,3.0,4.0,4.0,8.0,8.0
Displacement,313.0,196.442492,107.096439,68.0,105.0,151.0,302.0,455.0
Horsepower,313.0,106.246006,38.391040,46.0,76.0,95.0,130.0,225.0
Weight,313.0,2998.501597,860.658410,1613.0,2234.0,2800.0,3664.0,5140.0
Acceleration,313.0,15.346326,2.712730,8.0,13.5,15.2,17.0,24.8
Model Year,313.0,75.875399,3.635712,70.0,73.0,76.0,79.0,82.0
Origin,313.0,1.568690,0.794156,1.0,1.0,1.0,2.0,3.0


In [6]:
numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']

df_train_norm, df_test_norm = df_train.copy(), df_test.copy()

for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, 'mean']
    std  = train_stats.loc[col_name, 'std']
    df_train_norm.loc[:, col_name] = (df_train_norm.loc[:, col_name] - mean)/std
    df_test_norm.loc[:, col_name] = (df_test_norm.loc[:, col_name] - mean)/std
    
df_train_norm.tail()

/var/folders/n3/4qctfjhd43j9h6fypc3p1h480000gn/T/ipykernel_80422/4283327601.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.86591765  1.43096562  0.28252398 -0.86591765  0.28252398 -0.86591765
  1.43096562  0.28252398  1.43096562 -0.86591765  1.43096562 -0.86591765
  1.43096562 -0.86591765  1.43096562 -0.86591765  1.43096562 -0.86591765
  1.43096562 -0.86591765  0.28252398 -0.86591765 -0.86591765  1.43096562
 -0.86591765  1.43096562 -0.29169684 -0.86591765  1.43096562 -0.86591765
 -0.86591765 -0.86591765 -0.86591765  0.28252398  0.28252398  1.43096562
 -0.86591765 -0.86591765 -0.86591765 -0.86591765  1.43096562 -0.86591765
 -0.86591765 -0.86591765 -0.86591765 -0.86591765 -1.44013847 -0.86591765
 -0.86591765  1.43096562 -0.86591765 -0.86591765  1.43096562  1.43096562
  1.43096562 -0.86591765  0.28252398 -0.86591765  0.28252398 -0.86591765
 -0.86591765  0.28252398  0.28252398  1.43096562  0.28252398  0.282523

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
175,23.0,-0.865918,-0.760459,-0.292933,-0.353801,-0.127667,75,2
45,22.0,-0.865918,-0.527025,-0.892031,-0.686104,1.346862,71,1
130,25.0,-0.865918,-0.527025,-0.813888,-0.530410,0.609598,74,1
242,43.1,-0.865918,-0.993894,-1.517177,-1.177589,2.268443,78,2
172,18.0,0.282524,-0.237566,-0.240838,-0.016849,-0.311983,75,1


Grupujemy informacje o roku modelowym w następujący sposób:
$$
\text{bucket} =
\begin{cases}
0 & \text{jeśli year} < 73, \\
1 & \text{jeśli } 73 \leq \text{year} < 76, \\
2 & \text{jeśli } 76 \leq \text{year} < 79, \\
3 & \text{jeśli year} \geq 79.
\end{cases}
$$

In [7]:
boundaries = torch.tensor([73, 76, 79])
 
v = torch.tensor(df_train_norm['Model Year'].values)
df_train_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)

v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)

numeric_column_names.append('Model Year Bucketed')

Stosujemy one-hot-encoding do nieuporządkowanej zmiennej jakościowej Origin:

In [9]:
from torch.nn.functional import one_hot


total_origin = len(set(df_train_norm['Origin']))

origin_encoded = one_hot(torch.from_numpy(df_train_norm['Origin'].values) % total_origin)
x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()
 
origin_encoded = one_hot(torch.from_numpy(df_test_norm['Origin'].values) % total_origin)    ###########
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)                ###########
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float() #######


Tworzymy tensory z wartościami zmiennej zależnej MPG:

In [10]:
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()

---

Tworzymy DataLoader z `batch_size = 8` dla danych uczących (`shuffle=True`):

In [11]:
from torch.utils.data import DataLoader, TensorDataset

train_ds = TensorDataset(x_train, y_train)
batch_size = 8
torch.manual_seed(1)
train_dl = DataLoader(train_ds, batch_size, shuffle=True)   ###########

Budujemy sieć typu MLP z dwiema ukrytymi warstwami (rozmiaru 8 oraz 4):

In [12]:
hidden_units = [8, 4]
input_size = x_train.shape[1]

all_layers = []
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 1))

model = nn.Sequential(*all_layers)

model

Sequential(
  (0): Linear(in_features=9, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

Wybieramy funkcję straty MSE oraz SGD jako optymalizator (`lr=0.001` na początek):

In [13]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

Uczymy model przez 240 epok i wyświetlamy stratę na zbiorze uczącym co 20 epok:

In [16]:
num_epochs = 240
log_epochs = 20

for epoch in range(num_epochs):
    loss_hist_train = 0
    
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)[:, 0]
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        loss_hist_train += loss.item()
    if epoch % log_epochs==0:
        print(f'Epoch {epoch}  Loss {loss_hist_train/len(train_dl):.4f}')

Epoch 0  Loss 523.0650
Epoch 20  Loss 7.4822
Epoch 40  Loss 9.4665
Epoch 60  Loss 6.8254
Epoch 80  Loss 8.5104
Epoch 100  Loss 6.9554
Epoch 120  Loss 6.9052
Epoch 140  Loss 6.6565
Epoch 160  Loss 6.3265
Epoch 180  Loss 6.5258
Epoch 200  Loss 6.1354
Epoch 220  Loss 6.3239


Sprawdzamy jakość modelu (MSE oraz MAE) na zbiorze testowym:

In [17]:
with torch.no_grad():
    pred = model(x_test.float())[:, 0]
    loss = loss_fn(pred, y_test)
    print(f'Test MSE: {loss.item():.4f}')
    print(f'Test MAE: {nn.L1Loss()(pred, y_test).item():.4f}')

Test MSE: 9.6410
Test MAE: 2.2533
